In [1]:
import tensorflow as tf
import os
import zipfile
from os import path, getcwd, chdir

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/happy-or-sad")
zip_ref.close()

--2020-06-18 01:52:33--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 2607:f8b0:400e:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2020-06-18 01:52:34 (107 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [ ]:
# Directory with our training happy pictures
train_happy_dir = os.path.join('/tmp/happy-or-sad/happy')

# Directory with our training sad pictures
train_sad_dir = os.path.join('/tmp/happy-or-sad/sad')

print('total training happy images:', len(os.listdir(train_happy_dir)))
print('total training sad images:', len(os.listdir(train_sad_dir)))

total training happy images: 40
total training sad images: 40


In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
def train_happy_sad_model():
  DESIRED_ACCURACY = 0.999
  model =  tf.keras.Sequential([
          # first convolution
          tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
          tf.keras.layers.MaxPool2D(2,2),
          #first convolution
          tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
          tf.keras.layers.MaxPool2D(2,2),
          # third convolution
          tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
          tf.keras.layers.MaxPool2D(2,2),

          # Flatten the results to feed into a DNN
          tf.keras.layers.Flatten(),
          # 512 neuron hidden layer
          tf.keras.layers.Dense(512, activation='relu'),
          # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
          tf.keras.layers.Dense(1, activation='sigmoid')                              
  ])
  
  model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])
  
  # All images will be rescaled by 1./255
  train_datagen = ImageDataGenerator(rescale=1/255)
  validation_datagen = ImageDataGenerator(rescale=1/255)

  # Flow training images in batches of 128 using train_datagen generator
  train_generator = train_datagen.flow_from_directory(
        '/tmp/happy-or-sad/',  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150X150
        batch_size=10,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

  class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')> DESIRED_ACCURACY):
          print("\nReached 99.9% accuracy so cancelling training!")
          self.model.stop_training = True
  callbacks = myCallback()
  model.summary()
  history = model.fit(train_generator,steps_per_epoch=8, epochs=15,verbose=1,callbacks=[callbacks])
  return history.epoch,  history.history['accuracy'][-1]

In [ ]:
train_happy_sad_model()

Found 80 images belonging to 2 classes.
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_3 (Fla

([0, 1, 2, 3, 4, 5, 6], 1.0)